In [73]:
holding_stocks = {
    '000651':['格力电器', 0.1572, 46.00],
    '300347':['泰格医药', 0.0167, 74.20],
    '300326':['凯利泰  ', 0.0243, 18.74],
    '603882':['金域医学', 0.0125, 50.00],
    '600585':['海螺水泥', 0.0070, 40.00],
    '002821':['凯莱英  ', 0.0240, 227.0],
    '300529':['健帆生物', 0.0200, 56.30],
    '600835':['上海机电', 0.0320, 17.00],
}

attention_stocks = {
    '603719':['良品铺子', 0.1300, 14.98], # 高瓴买入时候为60亿的估值
    '603195':['公牛集团', 0.0200, 66.58], # 
    '688180':['君实生物', 0.01, 18],
    '603087':['甘李药业', 0.0452, 17.8],
    '600779':['水井坊', 0.01, 40],
    '601012':['隆基股份', 0, 70],
    '300015':['爱尔眼科',0,17],
    '600438':['通威股份',0, 29]
}

def code_fix(code):
    return code + '.SH' if code[0]=='6' else code + '.SZ'

  
from collections import ChainMap

# all_code = {code_fix(k):v for k,v in attention_stocks.items()}
all_code = {code_fix(k):v for k,v in holding_stocks.items()}

In [74]:
import pandas as pd
from pandas import DataFrame,Series
import tushare as ts
ts.set_token('61bd0479a5591b9a4c0112b0255ca10d29aeb1bfc2ea72cec0314725')
pro = ts.pro_api('61bd0479a5591b9a4c0112b0255ca10d29aeb1bfc2ea72cec0314725')
from datetime import datetime,timedelta
today_str = (datetime.now()- pd.tseries.offsets.DateOffset(days=5)).strftime("%Y%m%d")
pro = ts.pro_api()
Price = pro.daily(ts_code=','.join(all_code.keys()), adj='qfq',
                  start_date=today_str, fields='ts_code,trade_date,pre_close,close')
print(Price['trade_date'].max(), Price['trade_date'].min())
Price = Price.sort_values(['ts_code', 'trade_date'])
Price['TodayPct'] = Price['close'] / Price['pre_close'] - 1
Price = Price.groupby(['ts_code']).aggregate({'close':'last','TodayPct':'last'
                                              ,'pre_close':'first'}).reset_index()
Price['pre_close'] = Price['close'] / Price['pre_close'] - 1
Price.rename(columns={'ts_code':'Code', 'close':'Price1', 'pre_close':'WeekPct'},
             inplace=True)

20201225 20201221


In [79]:
sort_col = ['TodayPct', 'WeekPct'][-1]

df = DataFrame(Series(all_code)).reset_index()
for i in range(1,4):
    df[i] = df[0].map(lambda k:k[i-1])
df.drop(0, axis=1, inplace=True)
df.rename(columns = {'index':'Code',1:'Name', 2:'Hold_Ratio', 3:'Price0'}, inplace=True)

df = pd.merge(df, Price, on='Code')
df['Status'] = df[sort_col].map(lambda k: abs(int(round(100*k,0))) * ('+' if k>0 else '-'))

df = df[['Code','Name','Status','TodayPct','Price1','WeekPct','Price0','Hold_Ratio']]
df['Target'] = df['Price0']*3
df['Multiply'] = df['Price1']/df['Price0']
df.sort_values(by=sort_col, ascending=False, inplace=True)
df.index = range(len(df))
print(format(df['WeekPct'].mean(),'.2%'))
cols = ['Price0', 'Price1', 'Target','Multiply', 'WeekPct', 'Hold_Ratio', 'TodayPct']
for i, col in enumerate(cols):
    format_type = '0.2f' if i<=3 else '.2%'
    df[col] = df[col].map(lambda k: format(k, format_type)) 

df.style.set_properties(**{'text-align': 'right'})
df#.sort_values(by='Multiply', ascending=False)

5.81%


,Code,Name,Status,TodayPct,Price1,WeekPct,Price0,Hold_Ratio,Target,Multiply
2,603087.SH,甘李药业,++++++,0.93%,130.10,5.87%,17.80,4.52%,53.40,7.31
4,300015.SZ,爱尔眼科,+++++,0.14%,72.80,4.75%,17.00,0.00%,51.00,4.28
5,688180.SH,君实生物,,-1.11%,72.27,0.26%,18.00,1.00%,54.00,4.01
6,603719.SH,良品铺子,-,-3.16%,59.17,-1.38%,14.98,13.00%,44.94,3.95
3,603195.SH,公牛集团,+++++,2.82%,208.90,4.76%,66.58,2.00%,199.74,3.14
7,600779.SH,水井坊,---------,-2.89%,71.89,-8.76%,40.00,1.00%,120.00,1.80
0,600438.SH,通威股份,+++++++++++++++++++++,6.17%,40.58,20.63%,29.00,0.00%,87.00,1.40
1,601012.SH,隆基股份,++++++++++++++++++++,3.15%,93.45,20.35%,70.00,0.00%,210.00,1.33


In [ ]:
# import pandas as pd
# from pandas import DataFrame,Series
# import tushare as ts
# ts.set_token('61bd0479a5591b9a4c0112b0255ca10d29aeb1bfc2ea72cec0314725')
# pro = ts.pro_api('61bd0479a5591b9a4c0112b0255ca10d29aeb1bfc2ea72cec0314725')
# from datetime import datetime,timedelta
# today_str = (datetime.now()- pd.tseries.offsets.DateOffset(days=5)).strftime("%Y%m%d")
# pro = ts.pro_api()

# Price = pro.daily(ts_code=','.join(all_code.keys()), adj='qfq', start_date=today_str, fields='ts_code,trade_date,pre_close,close')
# Price = Price.sort_values(['ts_code', 'trade_date'])